In [17]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import os
import sys

In [18]:
import seaborn as sns
sns.set_style('white')
sns.set_palette('colorblind')

In [19]:
from imp import reload

In [40]:
def append_path(module_path):
    if module_path not in sys.path:
        sys.path.append(module_path)

append_path(os.path.abspath('..'))

from preprocess import essay_documents, process_arg_essays_for_conll

In [41]:
import read_annotations
reload(read_annotations);
reload(essay_documents);
reload(process_arg_essays_for_conll);

## Read annotations

In [42]:
document_pairs = read_annotations.get_annotated_documents()
document_pairs

[(Case: ALKASI v. TURKEY - Ann: S, Case: ALKASI v. TURKEY - Ann: M)]

## Align annotations

We consider two components aligned if they share 80% of the tokens

In [43]:
doc1, doc2 = document_pairs[0]

In [88]:
def are_aligned(interval1, interval2, plot=False):
    if interval1[0] < interval2[0]:
        start1, end1 = interval1
        start2, end2 = interval2
    elif interval1[0] > interval2[0]:
        start1, end1 = interval2
        start2, end2 = interval1
    else:  # They are equal, we need the interval 1 to be the shorter
        start1, end1 = interval1 if interval1[1] < interval2[1] else interval2
        start2, end2 = interval1 if interval1[1] >= interval2[1] else interval2
    overlap = end1 - start2
    if plot:
        plt.figure(figsize=(6, 2))
        sns.pointplot((interval1[0], interval2[0],interval1[1],interval2[1]),
                      (1,1,1,1), hue=['doc1', 'doc2', 'doc1', 'doc2'])
        plt.show()
    if overlap > (end1 - start1)*0.8 and overlap > (end2 - start2)*0.8:
        return True
    return False

def get_aligned_components(doc1, doc2):
    """Returns a list with components that are aligned between documents"""
    component_starts2 = sorted(doc2.annotated_components.keys())
    component_starts1 = sorted(doc1.annotated_components.keys())
    aligned_components = []
    # Go through the components of doc1 in ordered fashion.
    while len(component_starts1) is not 0 and len(component_starts2) is not 0:
        start1 = component_starts1[0]
        end1 = doc1.annotated_components[start1]
        start2 = component_starts2[0]
        end2 = doc2.annotated_components[start2]
        # Check if the first components in the stacks are aligned.
        if are_aligned((start1, end1), (start2, end2)):
            aligned_components.append((start1, start2))
            component_starts2.pop(0)
            component_starts1.pop(0)
        elif start1 >= start2:  # component2 can't be aligned
            component_starts2.pop(0)
        elif start1 <= start2:  # component2 can still be aligned, but not component1
            component_starts1.pop(0)
    return aligned_components

In [90]:
aligned_components = get_aligned_components(doc1, doc2)

Proportion of aligned components

In [99]:
for doc1, doc2 in document_pairs:
    print('[{0}] {1:.2f} - [{2}] {3:.2f}'.format(
        doc1.identifier.replace('Case: ', ''),
        len(aligned_components)/float(len(doc1.annotated_components)),
        doc2.identifier.replace('Case: ', ''),
        len(aligned_components)/float(len(doc2.annotated_components))))

[ALKASI v. TURKEY - Ann: S] 0.64 - [ALKASI v. TURKEY - Ann: M] 0.67
